# River input data
**Author: Jun Sasaki  Coded on 2025-05-10 Updated on 2025-05-10**<br>


In [ ]:
from pathlib import Path
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles
import matplotlib.pyplot as plt

from xfvcom import FvcomDataLoader
from xfvcom import FvcomPlotter, FvcomPlotConfig, FvcomPlotOptions
from xfvcom import parse_river_namelist
from xfvcom import make_node_marker_post
# from xfvcom.plot.markers_debug import make_node_marker_post_debug

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

png_dir = Path("PNG"); png_dir.mkdir(exist_ok=True)

In [ ]:
base_path = Path("~/Github/TB-FVCOM/goto2023").expanduser()
ncfile = "TokyoBay18_r16_crossed_0001.nc"
obcfile = "TokyoBay18_obc.dat"
# nc_river = "TokyoBayfinal_river.nc"
river_nc = "TokyoBayfinal16_river.nc"
river_nml = "RIVERS_NAMELIST16.nml" 
ncfile_path = base_path / "output" / ncfile
obcfile_path = base_path / "input" / obcfile
river_nc_path = base_path / "input" / river_nc
river_nml_path = base_path / "input" / river_nml
river = FvcomDataLoader(ncfile_path=river_nc_path, time_tolerance=5)
df = parse_river_namelist(river_nml_path)

fvcom = FvcomDataLoader(ncfile_path=ncfile_path, obcfile_path=obcfile_path, time_tolerance=5)
cfg = FvcomPlotConfig()
plotter = FvcomPlotter(fvcom.ds, cfg)

In [ ]:
pp = make_node_marker_post(
    df.grid_location,
    plotter,
    text_kwargs={"color": "cyan"},
    index_base=1,
)

#pp = make_node_marker_post_debug(
#    df.grid_location,   # pandas.Series[int]
#    plotter,
#    index_base=1,
#)

opts = FvcomPlotOptions(
    figsize=(6,9),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="#ffffff",
    mesh_linewidth=0.3,
    xlim = ("139:40:00", "140:00:00"),
    ylim = ("35:12:00", "35:42"),
    lon_tick_skip=2,
)

plotter.plot_2d(da=None, post_process_func=pp, opts=opts)

In [ ]:
# ------------------------------------------------------------
# Factory that creates a post_process_func capturing df/plotter
# ------------------------------------------------------------
def make_river_marker_post(df, plotter, *, show_name=True,
                           marker_kwargs=None, text_kwargs=None,
                           index_base: int = 0):
    """
    Return a post_process_func that plots river nodes on an existing axis.

    Parameters
    ----------
    df : pandas.DataFrame
        Output of ``parse_river_namelist`` with a ``grid_location`` column
        already 0-based.
    plotter : FvcomPlotter
        The plotter used for mesh plotting (to access lon/lat arrays).
    show_name : bool, default True
        Whether to write river names beside markers.
    marker_kwargs : dict | None
        Extra kwargs passed to ``ax.plot`` for the marker.
    text_kwargs : dict | None
        Extra kwargs passed to ``ax.text`` for the label.
    index_base : int, default 0
        Add this offset to the displayed node number (0→Python, 1→Fortran).
    """
    transform = ccrs.PlateCarree()
    mkw = {"marker": "o", "color": "red", "markersize": 3,
           "transform": transform, "zorder": 4} | (marker_kwargs or {})
    tkw = {"fontsize": 8, "color": "yellow", "ha": "center", "va": "bottom",
           "transform": transform, "zorder": 5, "clip_on": True} | (text_kwargs or {})

    lon = plotter.ds.lon.values     # cached for speed
    lat = plotter.ds.lat.values

    def _post(ax):                  # ← 実際に plot_2d が呼ぶ関数
        for _, row in df.iterrows():
            idx = int(row.grid_location)         # already 0-indexed
            ax.plot(lon[idx], lat[idx], **mkw)   # marker
            if show_name:
                label = f"{idx + index_base}"
                ax.text(lon[idx], lat[idx], label, **tkw)

    return _post

opts = FvcomPlotOptions(
    figsize=(6,9),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="#ffffff",
    mesh_linewidth=0.3,
    xlim = ("139:40:00", "140:00:00"),
    ylim = ("35:12:00", "35:42"),
    lon_tick_skip=2,
)

post_process_func = make_river_marker_post(df, plotter,
                                 show_name=True,
                                 index_base=1)
plotter.plot_2d(da=None, post_process_func=post_process_func, opts=opts)

In [ ]:
river.ds.river_names.values